In [ ]:
import numpy as np 

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Diamond

In [ ]:
output_notebook()

In [ ]:
x, y = [1, 2, 3, 4, 5], [6, 7, 2, 4, 5]

In [ ]:
# My First Scatter Plot:

# create a new plot with default tools, using figure
p = figure()

# add a circle renderer with x and y coordinates
p.circle(x, y)

# radius=0.4   - diameter to graph scale
# size=45
# 'vectorize' sizes: 

show(p) # show the results

In [ ]:
p = figure(plot_height=200, sizing_mode="scale_width")

p.title.text = 'First Plot'
p.background_fill_color = 'beige'
p.outline_line_width = 7
p.outline_line_alpha = 0.3
p.outline_line_color = "navy"

r = p.square(x, y, color="firebrick")

r.glyph.size = 50
r.glyph.fill_alpha = 0.2
r.glyph.line_color = "firebrick"
r.glyph.line_dash = [5, 1]
r.glyph.line_width = 2


show(p) # show the results

## Modify Axis Styling

In [ ]:
from math import pi
p = figure(plot_height=200, sizing_mode="scale_width", tools='')
p.square(x, y, size=50)

# change just some things about the x-axes
p.xaxis.axis_label = "Temp"
p.xaxis.axis_line_width = 3
p.xaxis.axis_line_color = "red"
p.xaxis.major_label_orientation = pi / 3

# change just some things about the y-axes
p.yaxis.axis_label = "Pressure"
p.yaxis.major_label_text_color = "orange"
p.yaxis.major_label_orientation = "vertical"
p.yaxis.axis_line_color = "#00ff00"

# change things on all axes
p.axis.minor_tick_in = -3
p.axis.minor_tick_out = 6

show(p) # show the results

## Selecting Glyphs

In [ ]:
p = figure(plot_height=200, sizing_mode="scale_width", tools='tap', title='Select a Square')
renderer = p.circle(x, y, 
                        size=75,
#                         radius=0.5, # GOT-CHA - Hit testing doesn't quite work with radius on Circles

                    # set visual properties for selected glyphs
                    selection_color="firebrick",

                    # set visual properties for non-selected glyphs
                    nonselection_fill_alpha=0.2,
                    nonselection_fill_color="grey",
                    nonselection_line_color="firebrick",
                    nonselection_line_alpha=1.0)

show(p)
# Hold Shift / Ctrl for multi-select

## Glyph Types:
https://bokeh.pydata.org/en/latest/docs/reference/models/markers.html

<img src="images/bokeh_models_markers.png" width="200"/>


## Hovering Over Data

In [ ]:
from bokeh.plotting import ColumnDataSource


source = ColumnDataSource(data=dict(
    x=x,
    y=y,
    desc=['A', 'b', 'C', 'd', 'E'],
))

TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y{0})"),
    ("desc", "@desc"),
]

p = figure(plot_width=400, plot_height=400, tooltips=TOOLTIPS,
           title="Mouse over the dots")

p.circle('x', 'y', size=40, source=source)

print(x)
print(y)
show(p)

In [ ]:
from bokeh.models.tools import HoverTool
from bokeh.sampledata.glucose import data

subset = data.loc['2010-10-06']

x, y = subset.index.to_series(), subset['glucose']

# Basic plot setup
p = figure(width=600, height=300, x_axis_type="datetime", title='Hover over points')

p.line(x, y, line_dash="4 4", line_width=1, color='gray')

cr = p.circle(x, y, size=20,
              fill_color="grey", hover_fill_color="firebrick",
              fill_alpha=0.05, hover_alpha=0.3,
              line_color=None, hover_line_color="white")

p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))

show(p)

In [ ]:
from bokeh.sampledata.stocks import AAPL
from bokeh.models import ColumnDataSource

tmp = AAPL
tmp['adj close'] = AAPL['adj_close']

tmp['date'] = np.array(AAPL['date'], dtype=np.datetime64) # convert date strings to real datetimes

p = figure(x_axis_type="datetime", title="AAPL", plot_height=450, sizing_mode='scale_width')
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=0.5
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Value'

p.line('date', 'adj close', source=ColumnDataSource(data=tmp), line_dash="dashed", line_color='grey')


ht = HoverTool(
    tooltips=[
        ( 'date',   '@date{%F}'            ),
        ( 'close',  '$@{adj close}{%0.2f}' ), # use @{ } for field names with spaces
        ( 'volume', '@volume{0.00 a}'      ),
    ],

    formatters={
        'date'      : 'datetime', # use 'datetime' formatter for 'date' field
        'adj close' : 'printf',   # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)
p.add_tools(ht)

show(p)
# len(tmp['date'])

## Annotations

In [ ]:
from bokeh.models.annotations import Arrow
from bokeh.models.arrow_heads import OpenHead, NormalHead, VeeHead # Arrow head Types
from bokeh.sampledata.stocks import AAPL
from bokeh.models import ColumnDataSource
from bokeh.models.annotations import BoxAnnotation


tmp = AAPL
tmp['adj close'] = AAPL['adj_close']

tmp['date'] = np.array(AAPL['date'], dtype=np.datetime64) # convert date strings to real datetimes

p = figure(x_axis_type="datetime", title="AAPL", plot_height=450, sizing_mode='scale_width')
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=0.5
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Value'

p.line('date', 'adj close', source=ColumnDataSource(data=tmp), line_dash="dashed", line_color='grey')

# x_start and _end need to be np.datetime64 data types to match the underlying data source
p.add_layout(Arrow(end=NormalHead(fill_color="red"),
                   x_start=np.datetime64('2000-11-14'), y_start=20, x_end=np.datetime64('2012-08-01'), y_end=680))

# region that always fills the top of the plot
upper = BoxAnnotation(bottom=600, fill_alpha=0.1, fill_color='olive')
p.add_layout(upper)

# region that always fills the bottom of the plot
lower = BoxAnnotation(top=200, fill_alpha=0.1, fill_color='black')
p.add_layout(lower)

# a finite region
center = BoxAnnotation(top=500, bottom=300, left=np.datetime64('2012-01-01'), right=np.datetime64('2014-01-01'), 
                       fill_alpha=0.1, fill_color='navy')
p.add_layout(center)

show(p)
# Can Also add lines, circles, elipsis, or other polygons

## Linking Graphs with ColumnDataSource